Data loading and initial exploration

Load the dataset into a pandas DataFrame
Explore the first few rows to understand the structure of data
Check the data types, summary statistics, and unique values of each column
Identify any data quality issues or inconsistencies

In [ ]:
import pandas as pd

df=pd.read_csv("../CaseStudy3/SuperStore_Dataset.csv")
# Display the first 10 rows of the DataFrame    
first_10_rows_data = df.head(10)
print(f"First 10 rows of data:\n {first_10_rows_data} \n")
# Display the last 10 rows of the DataFrame    
last_10_rows_data = df.tail(10)
print(f"Last 10 rows of data: \n {last_10_rows_data}")
#Display the data types of each column in the DataFrame
data_types_in_datasets=df.dtypes
print(f"Data types in the dataset:\n {data_types_in_datasets} \n")
#display summary statistics of the DataFrame
summary_statistics = df.describe()
print(f"Summary statistics of the dataset:\n {summary_statistics} \n")
#Display unique values in all columns of the DataFrame
unique_values = {col: df[col].unique() for col in df.columns}
print(f"Unique values in each column:\n {unique_values} \n")
#describe the DataFrame
data_information=df.info()
print(f"Data information:\n {data_information} \n")
#finding data quality issues
missing_values = df.isnull().sum()
print(f"Missing values in each column:\n {missing_values} \n")          
# Display the number of duplicate rows in the DataFrame
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows in the dataset: {duplicate_rows} \n")
# Display the number of rows and columns in the DataFrame
number_of_rows, number_of_columns = df.shape        
print(f"Number of rows: {number_of_rows}, Number of columns: {number_of_columns} \n")
# Display the column names in the DataFrame 
column_names = df.columns.tolist()
print(f"Column names in the dataset: {column_names} \n")    


Handling Duplicates

Identify and remove duplicate rows in the dataset
Document the number of rows and distinct Order IDs affected by this operation


In [ ]:
# 1. Check total rows before removing duplicates
total_rows_before = df.shape[0]
print(f"Total rows before removing duplicates: {total_rows_before}")

# 2. Find duplicates
duplicates = df[df.duplicated()]
num_duplicates = duplicates.shape[0]
print(f"Total duplicate rows: {num_duplicates}")

# 3. Check how many unique Order IDs are involved in these duplicate rows
affected_order_ids = duplicates['Order ID'].nunique()
print(f"Number of distinct Order IDs in duplicate rows: {affected_order_ids}")

# 4. Remove duplicate rows (keep first occurrence)
df_cleaned = df.drop_duplicates()

# 5. Verify new shape
total_rows_after = df_cleaned.shape[0]
print(f"Total rows after removing duplicates: {total_rows_after}")


Date Handling

Normalize the Order Date and Ship Date columns to ensure consistent date formats. Ensure that the format of the date is consistent across all rows
Extract the year from the Order ID and compare it with the year in the Order Date. Correct any inconsistencies
Document the number of rows and distinct Order IDs affected by these operations






In [ ]:
# Convert Order Date and Ship Date to datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce')
# Check for any rows where Order Date or Ship Date could not be converted
invalid_order_dates = df['Order Date'].isnull().sum()       
invalid_ship_dates = df['Ship Date'].isnull().sum()
print(f"Invalid Order Dates: {invalid_order_dates}")
print(f"Invalid Ship Dates: {invalid_ship_dates}")
# Check for any rows where Order Date is after Ship Date
invalid_date_order_ship = df[df['Order Date'] > df['Ship Date']]        
if not invalid_date_order_ship.empty:
    print(f"Rows with Order Date after Ship Date:\n{invalid_date_order_ship[['Order ID', 'Order Date', 'Ship Date']]} \n")  
else:
    print("No rows found with Order Date after Ship Date.\n")

#Extract the year from the Order ID and compare it with the year in the Order Date. Correct any inconsistencies
df['Order Year'] = df['Order ID'].str.extract(r'(\d{4})').astype(int)
df['Order Date Year'] = df['Order Date'].dt.year    
inconsistent_years = df[df['Order Year'] != df['Order Date Year']]
if not inconsistent_years.empty:
    print(f"Inconsistent years found:\n{inconsistent_years[['Order ID', 'Order Year', 'Order Date Year']]} \n") 
else:
    print("No inconsistencies found between Order ID year and Order Date year.\n")

#Document the number of rows and distinct Order IDs affected by these operations
affected_rows = inconsistent_years.shape[0]
affected_order_ids_count = inconsistent_years['Order ID'].nunique() 
print(f"Number of affected rows: {affected_rows}")
print(f"Number of distinct affected Order IDs: {affected_order_ids_count} \n")  

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv("../CaseStudy3/SuperStore_Cleaned_Dataset.csv", index=False)
print("Cleaned dataset saved to 'SuperStore_Cleaned_Dataset.csv'.") 






Imputation of Missing Values

Impute missing values in the Ship Mode column using the calculated Days to Ship column
Calculate Days to Ship as the difference between Ship Date and Order Date. If Days to Ship is 0, set Ship Mode to "Same Day"; if it is 7, set Ship Mode to "Standard Class"
Impute missing values in the Quantity column using a method of your choice. Print the rationale for selecting the imputation method
Document the number of rows and distinct Order IDs affected by these operations

In [ ]:
#Impute missing values in the Ship Mode column using the calculated Days to Ship column
df['Days to Ship'] = (df['Ship Date'] - df['Order Date']).dt.days
df['Days to Ship'] = df['Days to Ship'].fillna(df['Days to Ship'].mean())
# Impute missing values in the Ship Mode column
df['Ship Mode'] = df['Ship Mode'].fillna(df['Ship Mode'].mode()[0])

#Calculate Days to Ship as the difference between Ship Date and Order Date. If Days to Ship is 0, set Ship Mode to "Same Day"; if it is 7, set Ship Mode to "Standard Class"
df['Days to Ship'] = (df['Ship Date'] - df['Order Date']).dt.days
df.loc[df['Days to Ship'] == 0, 'Ship Mode'] = 'Same Day'
df.loc[df['Days to Ship'] == 7, 'Ship Mode'] = 'Standard Class'
print("Missing values in Ship Mode and Days to Ship have been imputed and updated accordingly.")
# Display the first few rows of the updated DataFrame
print(df.head())    
# Save the updated DataFrame to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_Updated_Dataset.csv", index=False)
print("Updated dataset saved to 'SuperStore_Updated_Dataset.csv'.")

#Impute missing values in the Quantity column using a method of your choice. Print the rationale for selecting the imputation method
df['Quantity'] = df['Quantity'].fillna(df['Quantity'].median())
print("Missing values in Quantity column have been imputed using the median value of the column, which is a robust method that minimizes the impact of outliers on the dataset.")   
# Display the first few rows of the DataFrame after imputation
print(df.head())
# Save the DataFrame with imputed Quantity to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_Quantity_Imputed_Dataset.csv", index=False)
print("Dataset with imputed Quantity saved to 'SuperStore_Quantity_Imputed_Dataset.csv'.")  

#Document the number of rows and distinct Order IDs affected by these operations
affected_rows_quantity = df[df['Quantity'].isnull()].shape[0]
affected_order_ids_quantity = df[df['Quantity'].isnull()]['Order ID'].nunique()
print(f"Number of affected rows in Quantity: {affected_rows_quantity}") 
print(f"Number of distinct affected Order IDs in Quantity: {affected_order_ids_quantity} \n")


Data Masking and String Handling

Drop the Customer Name column to protect Personal Identifiable Information (PII)
Create a new column called Customer Name Masked, containing only the initials of the customer name

Note: It's important to protect PII in datasets to maintain customer privacy and comply with data protection regulations. Masking or dropping sensitive data like customer names is a crucial step in this process
Convert the Postal Code column from numeric to text format, ensuring all codes are 5 characters long. Add a leading '0' where necessary


In [ ]:
# Create a new column called Customer Name Masked, containing only the initials of the customer name
df['Customer Name Masked'] = df['Customer Name'].apply(lambda x: ''.join([name[0] for name in x.split()]))
print("Customer Name Masked column has been created with initials of the customer names.")

# Drop the Customer Name column to protect Personal Identifiable Information (PII)
df = df.drop(columns=['Customer Name'])
print("Customer Name column has been dropped to protect Personal Identifiable Information (PII).")

# Convert the Postal Code column from numeric to text format, ensuring all codes are 5 characters long
df['Postal Code'] = df['Postal Code'].astype(str).str.zfill(5)
print("Postal Code column has been converted to text format with leading zeros where necessary.")

# Display the first few rows of the DataFrame after all modifications
print(df.head())

# Save the DataFrame with all modifications to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_Final_Dataset.csv", index=False)
print("Final dataset saved to 'SuperStore_Final_Dataset.csv'.")


Data Type Conversion

Convert the Quantity and Sales Price columns from strings to their appropriate numeric types (int and float, respectively)

In [ ]:
#Convert the Quantity and Sales Price columns from strings to their appropriate numeric types (int and float, respectively)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Sales Price'] = pd.to_numeric(df['Sales Price'], errors='coerce')



Handling Inconsistent Categorical Data

Clean the State column by replacing abbreviations with full state names (e.g., "CA" should be changed to "California"). You may need to research state abbreviations online to ensure all entries are corrected consistently

In [ ]:
#Clean the State column by replacing abbreviations with full state names (e.g., "CA" should be changed to "California"). You may need to research state abbreviations online to ensure all entries are corrected consistently
state_abbreviations = {
    'CA': 'California', 
    'NY': 'New York',
    'TX': 'Texas',      
    'FL': 'Florida',
    'IL': 'Illinois',
    'PA': 'Pennsylvania',
    'OH': 'Ohio',       
    'MI': 'Michigan',
    'NJ': 'New Jersey',
    'GA': 'Georgia',
    'NC': 'North Carolina',
    'VA': 'Virginia',
    'WA': 'Washington',
    'AZ': 'Arizona',
    'MA': 'Massachusetts',  
    'IN': 'Indiana',
    'TN': 'Tennessee',
    'MO': 'Missouri',
    'MD': 'Maryland',
    'WI': 'Wisconsin',
    'CO': 'Colorado',
    'MN': 'Minnesota',
    'SC': 'South Carolina',
    'AL': 'Alabama',
    'LA': 'Louisiana',
    'KY': 'Kentucky',
    'OR': 'Oregon', 
    'OK': 'Oklahoma',
    'CT': 'Connecticut',    
    'IA': 'Iowa',
    'KS': 'Kansas',
    'AR': 'Arkansas',
    'UT': 'Utah',
    'NV': 'Nevada',
    'MS': 'Mississippi',
    'NM': 'New Mexico',
    'WV': 'West Virginia',
    'NE': 'Nebraska',
    'ID': 'Idaho',
    'HI': 'Hawaii',
    'ME': 'Maine',
    'NH': 'New Hampshire',
    'RI': 'Rhode Island',
    'VT': 'Vermont',
    'DE': 'Delaware',
    'SD': 'South Dakota',
    'ND': 'North Dakota',
    'WY': 'Wyoming',
    'MT': 'Montana',
    'AK': 'Alaska',
    'DC': 'District of Columbia'
}
df['State'] = df['State'].replace(state_abbreviations)
print("State abbreviations have been replaced with full state names.")  
# Display the first few rows of the DataFrame after state name correction
print(df.head())
# Save the DataFrame with corrected state names to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_State_Corrected_Dataset.csv", index=False)
print("Dataset with corrected state names saved to 'SuperStore_State_Corrected_Dataset.csv'.")

    


Feature Engineering

Create new columns
Original Price: The price before any discount is applied
Total Sales: The total revenue is generated by multiplying the Sales Price by Quantity
Total Profit: The total profit is earned by multiplying the Profit by Quantity
Discount Price: The amount of discount applied, calculated based on the Original Price and Discount
Total Discount: The total discount value for the quantity sold
Create a new column, Shipping Urgency, based on Days to Ship
If Days to Ship is 0, set to "Immediate"
If Days to Ship is between 1 and 3, set to "Urgent"
If Days to Ship is more than 3, set to "Standard"
Create a column that calculates days since last order
Create a new dataset that stores the total sales, quantity, and discount per customer, and then merge these back to the original dataset

In [ ]:
'''Create new columns
Original Price: The price before any discount is applied
Total Sales: The total revenue is generated by multiplying the Sales Price by Quantity
Total Profit: The total profit is earned by multiplying the Profit by Quantity
Discount Price: The amount of discount applied, calculated based on the Original Price and Discount
Total Discount: The total discount value for the quantity sold'''

df['Original Price'] = df['Sales Price'] / (1 - df['Discount'])
df['Total Sales'] = df['Sales Price'] * df['Quantity']
df['Total Profit'] = df['Profit'] * df['Quantity']
df['Discount Price'] = df['Original Price'] * df['Discount']    
df['Total Discount'] = df['Discount Price'] * df['Quantity']
# Display the first few rows of the DataFrame after adding new columns
print(df.head())
# Save the DataFrame with new columns to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_New_Columns_Dataset.csv", index=False)
print("Dataset with new columns saved to 'SuperStore_New_Columns_Dataset.csv'.")



'''Create a new column, Shipping Urgency, based on Days to Ship
If Days to Ship is 0, set to "Immediate"
If Days to Ship is between 1 and 3, set to "Urgent"
If Days to Ship is more than 3, set to "Standard"'''
df['Shipping Urgency'] = 'Standard'  # Default value
df.loc[df['Days to Ship'] == 0, 'Shipping Urgency'] = 'Immediate'
df.loc[(df['Days to Ship'] > 0) & (df['Days to Ship'] <= 3), 'Shipping Urgency'] = 'Urgent'
# Display the first few rows of the DataFrame after adding Shipping Urgency
print(df.head())
# Save the DataFrame with Shipping Urgency to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_Shipping_Urgency_Dataset.csv", index=False)
print("Dataset with Shipping Urgency saved to 'SuperStore_Shipping_Urgency_Dataset.csv'.")

#Create a column that calculates days since last order
df['Days Since Last Order'] = (pd.to_datetime('today') - df['Order Date']).dt.days
# Display the first few rows of the DataFrame after adding Days Since Last Order
print(df.head())
# Save the DataFrame with Days Since Last Order to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_Days_Since_Last_Order_Dataset.csv", index=False)
print("Dataset with Days Since Last Order saved to 'SuperStore_Days_Since_Last_Order_Dataset.csv'.")

#Create a new dataset that stores the total sales, quantity, and discount per customer, and then merge these back to the original dataset
customer_summary = df.groupby('Customer ID').agg({
    'Total Sales': 'sum',
    'Quantity': 'sum',
    'Discount': 'mean'
}).reset_index()    
# Rename columns for clarity
customer_summary.columns = ['Customer ID', 'Total Sales per Customer', 'Total Quantity per Customer', 'Average Discount per Customer']
# Merge the customer summary back to the original DataFrame
df = df.merge(customer_summary, on='Customer ID', how='left')
# Display the first few rows of the DataFrame after merging customer summary
print(df.head())
# Save the DataFrame with customer summary to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_Customer_Summary_Dataset.csv", index=False)
print("Dataset with customer summary saved to 'SuperStore_Customer_Summary_Dataset.csv'.")


Outlier Detection and Handling

Identify and handle outliers in the Sales Price column
Create a function called remove_outliers that takes as an argument the dataframe and the column that needs to be searched for outliers. Using the 3 * IQR rule, the function should detect and remove the outliers to return the cleaned dataframe
Now use the function to detect outliers and remove outliers from the Sales Price and Profit columns

Why 3*IQR?
The 3 IQR method is applied in situations where the dataset has a high variance, and the standard 1.5 IQR might flag too many points as outliers. This method ensures that only the most extreme values are removed, preserving the integrity of the dataset while still mitigating the influence of true outliers

In [ ]:
#Identify and handle outliers in the Sales Price column
import numpy as np
from scipy import stats
# Calculate Z-scores for Sales Price
df['Sales Price Z-Score'] = np.abs(stats.zscore(df['Sales Price'], nan_policy='omit'))
# Define a threshold for outliers (e.g., Z-score > 3)   
outlier_threshold = 3
df['Is Outlier'] = df['Sales Price Z-Score'] > outlier_threshold
# Display the first few rows of the DataFrame after identifying outliers
print(df.head())
# Save the DataFrame with outlier information to a new CSV file
df.to_csv("../CaseStudy3/SuperStore_Outliers_Dataset.csv", index=False)
print("Dataset with outlier information saved to 'SuperStore_Outliers_Dataset.csv'.")

#Create a function called remove_outliers that takes as an argument the dataframe and the column that needs to be searched for outliers. Using the 3 * IQR rule, the function should detect and remove the outliers to return the cleaned dataframe
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    cleaned_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return cleaned_df
df_cleaned = remove_outliers(df, 'Sales Price')
# Display the first few rows of the cleaned DataFrame after removing outliers
print(df_cleaned.head())
# Save the cleaned DataFrame without outliers to a new CSV file
df_cleaned.to_csv("../CaseStudy3/SuperStore_Cleaned_Outliers_Dataset.csv", index=False)
print("Cleaned dataset without outliers saved to 'SuperStore_Cleaned_Outliers_Dataset.csv'.")


'''Now use the function to detect outliers and remove outliers from the Sales Price and Profit columns

Why 3*IQR?
The 3 IQR method is applied in situations where the dataset has a high variance, and the standard 1.5 IQR might flag too many points as outliers. This method ensures that only the most extreme values are removed, preserving the integrity of the dataset while still mitigating the influence of true outliers
'''

df_cleaned_sales_price = remove_outliers(df, 'Sales Price')
df_cleaned_profit = remove_outliers(df, 'Profit')
# Display the first few rows of the cleaned DataFrame after removing outliers from Sales Price
print(df_cleaned_sales_price.head())
# Save the cleaned DataFrame without outliers from Sales Price to a new CSV file
df_cleaned_sales_price.to_csv("../CaseStudy3/SuperStore_Cleaned_Sales_Price_Outliers_Dataset.csv", index=False)
print("Cleaned dataset without Sales Price outliers saved to 'SuperStore_Cleaned_Sales_Price_Outliers_Dataset.csv'.")
# Display the first few rows of the cleaned DataFrame after removing outliers from Profit
print(df_cleaned_profit.head())
# Save the cleaned DataFrame without outliers from Profit to a new CSV file
df_cleaned_profit.to_csv("../CaseStudy3/SuperStore_Cleaned_Profit_Outliers_Dataset.csv", index=False)
print("Cleaned dataset without Profit outliers saved to 'SuperStore_Cleaned_Profit_Outliers_Dataset.csv'.")


Customer Segmentation and Analysis

Calculate Customer Sales Quintile and Customer Profit Quintile based on total sales and total profit per Customer ID
What is a Quintile? Quintiles are a statistical way of dividing data into five equal parts, each representing 20% of the data. For example, customers in the top quintile (Q5) represent the top 20% of sales or profit
Create a cross-grid (cross-tabulation) based on these two quintiles to analyze the relationship between customer sales and profitability


In [ ]:
# Aggregate total sales and profit per customer
customer_summary = df.groupby('Customer ID').agg({
    'Total Sales': 'sum',
    'Total Profit': 'sum'
}).reset_index()

# Calculate quintiles on the aggregated data
customer_summary['Total Sales Quintile'] = pd.qcut(customer_summary['Total Sales'], 5, labels=[1, 2, 3, 4, 5])
customer_summary['Total Profit Quintile'] = pd.qcut(customer_summary['Total Profit'], 5, labels=[1, 2, 3, 4, 5])

# Merge quintile info back to the main DataFrame
df = df.merge(customer_summary[['Customer ID', 'Total Sales Quintile', 'Total Profit Quintile']], on='Customer ID', how='left')

# Display the first few rows
print(df.head())

# Save the DataFrame with quintiles
df.to_csv("../CaseStudy3/SuperStore_Quintiles_Dataset.csv", index=False)
print("Dataset with quintiles saved to 'SuperStore_Quintiles_Dataset.csv'.")

# Create cross-grid
cross_grid = pd.crosstab(df['Total Sales Quintile'], df['Total Profit Quintile'], margins=True, margins_name='Total')
print("Cross-grid of Customer Sales Quintile and Customer Profit Quintile:")
print(cross_grid)
cross_grid.to_csv("../CaseStudy3/SuperStore_Cross_Grid_Quintiles.csv")
print("Cross-grid saved to 'SuperStore_Cross_Grid_Quintiles.csv'.")

Final Analysis and Dashboard Creation

Sales and Profit Analysis

Top 10 Most Profitable Products: Use a bar chart to display the products with the highest total profit
Top 10 Most Loss-Making Products: Use a bar chart to display the products with the highest total losses (negative profit)
Sales vs. Profit Correlation: Use a scatter plot to visualize the correlation between Total Sales and Total Profit. Add a regression line to show the trend
Joint Distribution of Sales and Profit: Create a joint plot to analyze the relationship between Total Sales and Total Profit across different products
Customer Segmentation and Analysis

Customer Sales Quintile vs. Customer Profit Quintile: Create a heatmap or cross-tabulation to explore the relationship between customers' sales and profit quintiles. This will help identify which segments of customers are most valuable
Understand how different product categories perform across customer segments: Create a pivot table to analyze the total Sales and total Profit by Category and Segment. Sort the pivot table to highlight the most profitable and least profitable combinations of Category and Segment
Shipping and Delivery Analysis

Distribution of Shipping Urgency: Visualize the distribution of orders by Shipping Urgency using a pie chart or bar chart
Days to Ship vs. Profit: Use a violin plot to explore the distribution of Profit across different Days to Ship categories. This will help analyze whether faster shipping correlates with higher or lower profitability
Shipping Mode and Profitability: Create a grouped bar chart to compare the profitability of different shipping modes (e.g., Standard Class, First Class)
Using a pivot table, determine which shipping modes are most preferred across different regions and analyze the impact on total sales and profit. Create a pivot table that shows the count of Order IDs, total Sales, and total Profit for each Region and Ship Mode. Identify and print your insights
Regional Sales and Profitability

Sales and Profit by Region: Use a map or bar chart to visualize total sales and profit by region or state. This will highlight which regions are the most profitable

State-wise Profitability: Create a pivot table to summarize the profitability of each state. Highlight the top and bottom states based on profitability

Correlation between State and Profit: Use a correlation plot to identify any patterns or relationships between the states and the profitability of orders

(Hint: Convert the categorical 'State' column into numerical values using label encoding using “from sklearn.preprocessing import LabelEncoder”)

Discount and Pricing Analysis

Impact of Discounts on Profitability: Use a scatter plot with a trend line to analyze how different levels of discount affect profitability
Original Price vs. Discounted Price: Create a line plot to compare the original price and the discounted price across various product categories or sub-categories
Temporal Analysis

Sales and Profit Trends Over Time: Use a time series plot to analyze how sales and profit have trended over the years or months. This will help in identifying any seasonal patterns
Order Frequency by Month: Use a bar chart or line plot to show the number of orders placed each month. Highlight any months with unusually high or low order frequencies
Yearly Growth in Sales and Profit: Use a year-over-year growth chart to compare the sales and profit growth over different years

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# Top 10 Most Profitable Products
top_10_profitable_products = df.groupby('Product Name')['Total Profit'].sum().nlargest(10)
plt.figure(figsize=(12, 6))
top_10_profitable_products.plot(kind='bar', color='skyblue')
plt.title('Top 10 Most Profitable Products')
plt.xlabel('Product Name')
plt.ylabel('Total Profit')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Top 10 Most Loss-Making Products
top_10_loss_making_products = df.groupby('Product Name')['Total Profit'].sum().nsmallest(10)
plt.figure(figsize=(12, 6))
top_10_loss_making_products.plot(kind='bar', color='salmon')
plt.title('Top 10 Most Loss-Making Products')
plt.xlabel('Product Name')
plt.ylabel('Total Profit')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Sales vs. Profit Correlation
plt.figure(figsize=(10, 6))
plt.scatter(df['Total Sales'], df['Total Profit'], alpha=0.5, color='purple')
plt.title('Sales vs. Profit Correlation')
plt.xlabel('Total Sales')
plt.ylabel('Total Profit')
sns.regplot(x='Total Sales', y='Total Profit', data=df, scatter=False, color='red')
plt.tight_layout()
plt.show()

# Joint Distribution of Sales and Profit
g = sns.jointplot(x='Total Sales', y='Total Profit', data=df, kind='scatter', color='green')
g.fig.suptitle('Joint Distribution of Sales and Profit', y=1.02)
plt.show()

# Customer Sales Quintile vs. Customer Profit Quintile Heatmap
cross_tab_quintiles = pd.crosstab(df['Total Sales Quintile'], df['Total Profit Quintile'], margins=True, margins_name='Total')
plt.figure(figsize=(10, 8))
sns.heatmap(cross_tab_quintiles, annot=True, fmt='d', cmap='YlGnBu', cbar_kws={'label': 'Count'})
plt.title('Customer Sales Quintile vs. Customer Profit Quintile')
plt.xlabel('Customer Profit Quintile')
plt.ylabel('Customer Sales Quintile')
plt.tight_layout()
plt.show()

# Pivot Table: Total Sales and Profit by Category and Segment
pivot_table = df.pivot_table(values=['Total Sales', 'Total Profit'], index='Category', columns='Segment', aggfunc='sum')
pivot_table = pivot_table.sort_values(by=('Total Profit', 'Consumer'), ascending=False)
print("Pivot Table of Total Sales and Total Profit by Category and Segment:")
print(pivot_table)
pivot_table.to_csv("../CaseStudy3/SuperStore_Pivot_Table_Category_Segment.csv")
print("Pivot table saved to 'SuperStore_Pivot_Table_Category_Segment.csv'.")

# Distribution of Shipping Urgency
shipping_urgency_counts = df['Shipping Urgency'].value_counts()
plt.figure(figsize=(8, 6))
shipping_urgency_counts.plot(kind='bar', color='lightcoral')
plt.title('Distribution of Shipping Urgency')
plt.xlabel('Shipping Urgency')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Days to Ship vs. Profit (Violin Plot)
plt.figure(figsize=(10, 6))
sns.violinplot(x='Days to Ship', y='Total Profit', data=df, palette='muted')
plt.title('Days to Ship vs. Profit Distribution')
plt.xlabel('Days to Ship')
plt.ylabel('Total Profit')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Shipping Mode and Profitability
shipping_mode_profit = df.groupby('Ship Mode')['Total Profit'].sum().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
shipping_mode_profit.plot(kind='bar', color='teal')
plt.title('Shipping Mode and Profitability')
plt.xlabel('Shipping Mode')
plt.ylabel('Total Profit')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Pivot Table: Shipping Modes by Region
pivot_table_shipping_region = df.pivot_table(
    values=['Order ID', 'Total Sales', 'Total Profit'],
    index='Region',
    columns='Ship Mode',
    aggfunc={'Order ID': 'count', 'Total Sales': 'sum', 'Total Profit': 'sum'}
)
print("Pivot Table of Order Count, Total Sales, and Total Profit by Region and Ship Mode:")
print(pivot_table_shipping_region)
pivot_table_shipping_region.to_csv("../CaseStudy3/SuperStore_Pivot_Table_Shipping_Region.csv")
print("Pivot table saved to 'SuperStore_Pivot_Table_Shipping_Region.csv'.")

print("\nInsights from the pivot table:")
pt_dict = pivot_table_shipping_region.to_dict()
for region in pivot_table_shipping_region.index:
    for ship_mode in pivot_table_shipping_region.columns.get_level_values(0).unique():
        order_count = pt_dict.get((ship_mode, 'Order ID'), {}).get(region, 0)
        total_sales = pt_dict.get((ship_mode, 'Total Sales'), {}).get(region, 0)
        total_profit = pt_dict.get((ship_mode, 'Total Profit'), {}).get(region, 0)
        print(f"In {region}, the {ship_mode} shipping mode has {order_count} orders with total sales of ${total_sales:.2f} and total profit of ${total_profit:.2f}.")

affected_rows_shipping = df[df['Ship Mode'].isnull()].shape[0]
affected_order_ids_shipping = df[df['Ship Mode'].isnull()]['Order ID'].nunique()
print(f"Number of affected rows in Ship Mode: {affected_rows_shipping}")
print(f"Number of distinct affected Order IDs in Ship Mode: {affected_order_ids_shipping} \n")

# State-wise Profitability
state_profitability = df.groupby('State')['Total Profit'].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
state_profitability.plot(kind='bar', color='lightgreen')
plt.title('State-wise Profitability')
plt.xlabel('State')
plt.ylabel('Total Profit')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Correlation between State and Profit (Label Encoding)
df['State_encoded'] = LabelEncoder().fit_transform(df['State'])
correlation_matrix = df[['State_encoded', 'Total Profit']].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation between State and Profit')
plt.tight_layout()
plt.show()

# Impact of Discounts on Profitability
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Discount', y='Total Profit', data=df, alpha=0.5, color='orange')
sns.regplot(x='Discount', y='Total Profit', data=df, scatter=False, color='red')
plt.title('Impact of Discounts on Profitability')
plt.xlabel('Discount')
plt.ylabel('Total Profit')
plt.tight_layout()
plt.show()

# Original Price vs. Discounted Price by Category
plt.figure(figsize=(12, 6))
sns.lineplot(x='Original Price', y='Sales Price', data=df, hue='Category', palette='Set1')
plt.title('Original Price vs. Discounted Price by Category')
plt.xlabel('Original Price')
plt.ylabel('Sales Price')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Sales and Profit Trends Over Time
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
monthly_trends = df.groupby(df['Order Date'].dt.to_period('M')).agg({'Total Sales': 'sum', 'Total Profit': 'sum'}).reset_index()
monthly_trends['Order Date'] = monthly_trends['Order Date'].dt.to_timestamp()
plt.figure(figsize=(14, 7))
plt.plot(monthly_trends['Order Date'], monthly_trends['Total Sales'], label='Total Sales', color='blue')
plt.plot(monthly_trends['Order Date'], monthly_trends['Total Profit'], label='Total Profit', color='green')
plt.title('Sales and Profit Trends Over Time')
plt.xlabel('Date')
plt.ylabel('Amount')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Order Frequency by Month
monthly_order_frequency = df['Order Date'].dt.to_period('M').value_counts().sort_index()
plt.figure(figsize=(12, 6))
monthly_order_frequency.plot(kind='bar', color='purple')
plt.title('Order Frequency by Month')
plt.xlabel('Month')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Yearly Growth in Sales and Profit
yearly_growth = df.groupby(df['Order Date'].dt.year).agg({'Total Sales': 'sum', 'Total Profit': 'sum'}).reset_index()
yearly_growth.rename(columns={'Order Date': 'Year'}, inplace=True)
plt.figure(figsize=(12, 6))
plt.plot(yearly_growth['Year'], yearly_growth['Total Sales'], label='Total Sales', marker='o', color='blue')
plt.plot(yearly_growth['Year'], yearly_growth['Total Profit'], label='Total Profit', marker='o', color='green')
plt.title('Yearly Growth in Sales and Profit')
plt.xlabel('Year')
plt.ylabel('Amount')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()